<a href="https://colab.research.google.com/github/thirayume/muangtai/blob/main/Muangtai_PO_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **V.2 - Muangtai Purchasing Prediction by Month**

In [ ]:
# @title Define secrets

from google.colab import userdata
host = userdata.get('host')
port = userdata.get('port')
database = userdata.get('database')
user = userdata.get('user')
password = userdata.get('password')

In [ ]:
# @title Install dependencies (if need)

!pip install psycopg2 pandas pmdarima

In [ ]:
# @title Import dependencies

import psycopg2

from datetime import datetime

import numpy as np
import pandas as pd
from pandas import plotting

import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.pylab import rcParams
import seaborn as sns

import plotly as py
import plotly.graph_objs as go
py.offline.init_notebook_mode(connected = True)

import fastai

import warnings
import os
from pathlib import Path
warnings.filterwarnings("ignore")

import torch
import torch.nn as nn
from torch.autograd import Variable

import pmdarima as pm

from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

from math import sqrt

# plt.style.use('fivethirtyeight')
plt.style.use("seaborn-whitegrid")
plt.rc("figure", autolayout = True)
plt.rc("axes", labelweight = "bold", labelsize = "large", titleweight = "bold", titlesize = 12, titlepad = 10)

In [ ]:
# @title Connect to the PostgreSQL database

conn = psycopg2.connect(
    host=host,
    port=port,
    database=database,
    user=user,
    password=password,
)
conn

In [ ]:
# @title Define a SQL to create a "view_picking_list" (if not found)

sql = """
      -- STEP 1: create function "multiply_hierarchy_unit_quantity function"
      CREATE OR REPLACE FUNCTION multiply_hierarchy_unit_quantity(start_stock_keeping_unit_id uuid)
      RETURNS numeric AS $$
      DECLARE is_base_unit bool = false;
      DECLARE total numeric(12,8) = 1;
      BEGIN

          SELECT a.is_base_unit INTO is_base_unit FROM dim_stock_keeping_units a where a.id = start_stock_keeping_unit_id;

          if is_base_unit then
              SELECT 1 INTO total FROM dim_bill_of_materials where stock_keeping_unit_id = start_stock_keeping_unit_id;
          else
              WITH RECURSIVE unit_cte AS (
                  SELECT stock_keeping_unit_id, part_id, multiplier
                  FROM dim_bill_of_materials
                  where stock_keeping_unit_id = start_stock_keeping_unit_id
                  UNION ALL
                  SELECT h.stock_keeping_unit_id, h.part_id, h.multiplier
                  FROM dim_bill_of_materials h
                  INNER JOIN unit_cte cte ON h.stock_keeping_unit_id = cte.part_id
              )
              SELECT product(multiplier) INTO total FROM unit_cte;
          end if;
          RETURN total;
      END;
      $$ LANGUAGE plpgsql;

      -- Helper function to calculate product in a recursive query
      CREATE AGGREGATE product(numeric) (
          SFUNC = numeric_mul,
          STYPE = numeric,
          INITCOND = '1'
      );

      -- Multiplication function
      CREATE OR REPLACE FUNCTION numeric_mul(numeric, numeric)
      RETURNS numeric AS $$
      BEGIN
          RETURN $1 * $2;
      END;
      $$ LANGUAGE plpgsql;

      ----------------------
      -- STEP 2: Create view "view_dim_picking_lists_lasted_updated_at"
      DROP VIEW IF EXISTS view_dim_picking_lists_lasted_updated_at;

      CREATE OR REPLACE VIEW view_dim_picking_lists_lasted_updated_at
      AS
      select id,
          CASE
              WHEN a.document_date < (a.moderated_at::date)
                  THEN (a.document_date + '08:30:01'::time)
              -- IN CASE a.document_date >= a.moderated.date
              WHEN a.moderated_at is null and a.document_date is not null
                  THEN (a.document_date + '08:30:01'::time)
              WHEN a.moderated_at is null and a.document_date is null
                  THEN a.updated_at
              ELSE a.moderated_at
          END as date
      from dim_picking_lists a
      where moderation_status = 1;

      ----------------------
      -- create view "view_fact_picking_lists"
      DROP VIEW IF EXISTS view_fact_picking_lists;

      CREATE OR REPLACE VIEW view_fact_picking_lists
      AS

      select
      fact_picking_lists.id as fact_picking_lists__id
      ,to_char(view_dim_picking_lists_lasted_updated_at.date ::date, 'YYYY-MM-DD' ) as dim_pickinglists__document_date
      ,to_char(view_dim_picking_lists_lasted_updated_at.date ::time, 'HH24:MI:SS' ) as dim_pickinglists__document_time
      ,dim_picking_lists.document_number as dim_picking_lists__document_number
      ,dim_picking_lists.id as dim_picking_lists__id
      ,dim_stock_keeping_units.id as dim_stock_keeping_units__sku_id
      ,fact_picking_lists.quantity as fact_picking_lists__quantity
      , (fact_picking_lists.quantity  * multiply_hierarchy_unit_quantity(fact_picking_lists.stock_keeping_unit_id)) as liters
      ,dim_families.is_alcoholic as dim_families__is_alcoholic
      ,dim_picking_lists.owner_id as dim_picking_lists__owner_id
      ,dim_picking_lists.owner_type as dim_picking_lists__owner_type
      from		fact_picking_lists
      inner join	dim_dates on dim_dates.id = fact_picking_lists.date_id
      inner join	dim_times on dim_times.id = fact_picking_lists.time_id
      inner join	dim_families on dim_families.id = fact_picking_lists.family_id
      inner join	dim_brands on dim_brands.id = dim_families.brand_id
      inner join 	dim_picking_lists on dim_picking_lists.id = fact_picking_lists.picking_list_id
      inner join 	dim_stock_keeping_units on dim_stock_keeping_units.id = fact_picking_lists.stock_keeping_unit_id
      inner join  view_dim_picking_lists_lasted_updated_at on dim_picking_lists.id = view_dim_picking_lists_lasted_updated_at.id
      where fact_picking_lists.deleted_at is null
      and dim_picking_lists.moderation_status = 1 -- Approved
      and fact_picking_lists.direction = 'outbound'
      order by  fact_picking_lists.id desc;
      """

In [ ]:
# @title Define a SQL query to select data from a "fact_picking_lists" like in a View

sql = """
      select * from view_fact_picking_lists;
      """

In [ ]:
# @title Create a cursor object to execute queries to dataframe

with conn.cursor() as cursor:
  # Execute the query
  cursor.execute(sql)
  columns = [desc[0] for desc in cursor.description]
  # Fetch all rows from the query result
  rows = cursor.fetchall()

  # Create dataframe
  df = pd.DataFrame(rows, columns=columns)

In [ ]:
# @title Close the connection
conn.close()

In [ ]:
# @title Format Date and Time and Index

df['fact_picking_lists__podatetime'] = pd.to_datetime( df['fact_picking_lists__date']  + ' ' + df['fact_picking_lists__time'], infer_datetime_format=True)
df['lasted_updated_at'] = pd.to_datetime(df['lasted_updated_at'], infer_datetime_format = True)
df['fact_picking_lists__date'] = pd.to_datetime(df['lasted_updated_at'], infer_datetime_format = True)

In [ ]:
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

In [ ]:
df.head(5)

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
sku_ids = df['dim_stock_keeping_units__sku_id'].unique()
print(sorted(sku_ids))

In [ ]:
# @title Pivot SKU Sales by Month
pd.set_option("display.float_format", "{:,.0f}".format)

# df['month'] = df['fact_picking_lists__date'].dt.month.astype(str) + '-' + df['fact_picking_lists__date'].dt.year.astype(str)
# df['month'] = df['fact_picking_lists__podatetime'].dt.month.astype(str) + '-' + df['fact_picking_lists__podatetime'].dt.year.astype(str)
df['month'] = df['lasted_updated_at'].dt.month.astype(str) + '-' + df['lasted_updated_at'].dt.year.astype(str)
df['month'] = pd.to_datetime(df['month'], infer_datetime_format = True)

pivot = df.pivot_table(
    values='fact_picking_lists__quantity',
    index=['dim_stock_keeping_units__sku_id'],
    columns=['month'],
    aggfunc='sum',
    margins = True,
    fill_value = '0'
)
pivot = pivot.sort_values(by=['All'], ascending=False)
pivot

In [ ]:
pivot.T

In [ ]:
sorted_sku = []
for sku, qty in pivot.iterrows():
  sorted_sku.append(sku)
sorted_sku.pop(0)

In [ ]:
def plot_by_SKUs(sku_df):
  for i in range(len(sku_df)):
    print(sku_df[i])
    plt.figure(figsize=(6,3))
    my_df = pivot.T[sku_df[i]]
    my_df.drop(my_df.tail(1).index,inplace=True)
    my_df.astype(float).plot()
    plt.title(sku_df[i])
    plt.tight_layout()
    plt.show()

In [ ]:
sample = sorted_sku[0:1]
plot_by_SKUs(sample)

In [ ]:
def mape(actual, pred):
    actual, pred = np.array(actual), np.array(pred)
    return np.mean(np.abs((actual - pred) / actual)) * 100

In [ ]:
sample_sku = sorted_sku[0:1]
sample_sku

In [ ]:
sample_df = pivot.T[sample_sku].reset_index()
sample_df.drop(sample_df.tail(1).index, inplace=True)
sample_df.columns = ['month', 'qty']
sample_df['month'] = pd.to_datetime(sample_df['month'], infer_datetime_format = True)
sample_df = sample_df.set_index(['month'])
sample_df

In [ ]:
sample_df.info()

In [ ]:
plt.figure(figsize = (15, 7))
plt.title("Number of Purchase by Date")
plt.xlabel('Month')
plt.ylabel('QTY')
plt.plot(sample_df)
plt.show()

### <center> Rolling Statistics

A rolling average is a great way to visualize how the dataset is trending. As the dataset provides counts by month, a window size of 12 will give the annual rolling average.

this plot include the rolling standard deviation to see how much the data varies from the rolling average.

In [ ]:
#Determine rolling statistics
sample_df["rolling_avg"] = sample_df["qty"].rolling(window = 12).mean() #window size 12 denotes 12 months, giving rolling mean at yearly level
sample_df["rolling_std"] = sample_df["qty"].rolling(window = 12).std()

#Plot rolling statistics
plt.figure(figsize = (15, 7))
plt.plot(sample_df["qty"], color = '#379BDB', label = 'Original')
plt.plot(sample_df["rolling_avg"], color = '#D22A0D', label = 'Rolling Mean')
plt.plot(sample_df["rolling_std"], color = '#142039', label = 'Rolling Std')
plt.legend(loc = 'best')
plt.title('Rolling Mean & Standard Deviation')
plt.show(block = False)

### <center>SARIMA Model Selection</center>

Now let's try the same strategy as above, except let's use a SARIMA model so that can be account for seasonality.

In [ ]:
# sample_df = pivot.T[sample_sku].reset_index()
# sample_df.drop(sample_df.tail(1).index, inplace=True)
# sample_df.columns = ['month', 'qty']
# sample_df['month'] = pd.to_datetime(sample_df['month'], infer_datetime_format = True)
# sample_df = sample_df.set_index(['month'])
# sample_df

In [ ]:
# result = seasonal_decompose(sample_df, model='additive', extrapolate_trend='freq', period=3)
result = seasonal_decompose(sample_df, period=3)
fig = plt.figure()
fig = result.plot()

In [ ]:
result = adfuller(sample_df.qty.dropna())
print(f'ADF Statistics:{result[0]}')
print(f'p-value:{result[1]}')

The p-value is higher than 0.05. This means that the time serie is non stationary with a confidence of 95%. Then check if with a one step differentiation, the time serie become stationary (in terms of a trendless time series).

In [ ]:
result2 = adfuller(sample_df.qty.diff().dropna())
print(f'ADF Statistics:{result2[0]}')
print(f'p-value:{result2[1]}')

In [ ]:
# @title ACF and PACF

fig, (ax1, ax2) = plt.subplots(2, 1, figsize = (8, 8))

plot_acf(sample_df, lags = 7, zero = False, ax = ax1)
plot_pacf(sample_df, lags = 7, zero = False, ax = ax2)
plt.show()

In [ ]:
# Seasonal - fit stepwise auto-ARIMA
SARIMA_model = pm.auto_arima(sample_df, start_p = 1, start_q = 1,
                        max_p = 3, max_q = 3,
                        m = 6, # 6 is the half frequncy of the cycle
                        seasonal = True, # set to seasonal
                        d = 1,
                        D = 1, # order of the seasonal differencing
                        trace = True,
                        error_action = 'ignore',
                        start_P = 1, start_Q = 1,
                        max_P = 2, max_Q = 2,
                        information_criterion = 'aic',
                        stepwise = True)

### SARIMA (0,1,1)(0,1,0) [6] : AIC=191.035

In [ ]:
model = SARIMAX(sample_df.astype(float), order = (0, 1, 1), seasonal_order = (0, 1, 0, 6))
SARIMA_model = model.fit()

In [ ]:
SARIMA_model.summary()

In [ ]:
# @title Create the 4 diagostics plots
SARIMA_model.plot_diagnostics(figsize = (8, 8))
plt.show()

In [ ]:
# All the 4 plots indicates a good fit of the SARIMA model on the given time serie.
# Create a vector that will host the predictions
prediction = SARIMA_model.get_prediction(start = -12)
mean_prediction = prediction.predicted_mean
mean_prediction = mean_prediction.rename("prediction")

In [ ]:
# Get the confidence intervals from the sarima prediction
confi_int_p = prediction.conf_int()
lower_limits_p = confi_int_p.iloc[:, 0]
upper_limits_p = confi_int_p.iloc[:, 1]

In [ ]:
plt.figure(figsize = (14, 5))
plt.title("Purchase prediction by SARIMA", fontsize = 25)

plt.plot(sample_df[-24:].index, sample_df[-24:].values, label = 'Actual values', color = "blue", marker = "o")

plt.plot(mean_prediction[-24:].index, mean_prediction[-24:].values, label = 'Prediction', color = "green", marker = "o")
plt.fill_between(mean_prediction[-24:].index, lower_limits_p, upper_limits_p, alpha = 0.1, color = "green")

plt.legend(fontsize = 12, fancybox = True, shadow = True, frameon = True)
plt.ylabel('QTY', fontsize = 15)
plt.show()

In [ ]:
mape_sarima = mape(sample_df.iloc[-12:, 0], mean_prediction)
print(f"MAPE OF LSTM MODEL : {mape_sarima:.2f} %")

In [ ]:
rmse_sarima = sqrt(mean_squared_error(sample_df[-12:].values, mean_prediction.values))
print(f"RMSE OF LSTM MODEL : {rmse_sarima:.2f}")

### SARIMA(0,1,2)(0,1,0) [6] : AIC=191.828

In [ ]:
model = SARIMAX(sample_df.astype(float), order = (0, 1, 2), seasonal_order = (0, 1, 0, 6))
SARIMA_model = model.fit()

In [ ]:
SARIMA_model.summary()

In [ ]:
# @title Create the 4 diagostics plots
SARIMA_model.plot_diagnostics(figsize = (8, 8))
plt.show()

In [ ]:
# All the 4 plots indicates a good fit of the SARIMA model on the given time serie.
# Create a vector that will host the predictions
prediction = SARIMA_model.get_prediction(start = -12)
mean_prediction = prediction.predicted_mean
mean_prediction = mean_prediction.rename("prediction")

In [ ]:
# Get the confidence intervals from the sarima prediction
confi_int_p = prediction.conf_int()
lower_limits_p = confi_int_p.iloc[:, 0]
upper_limits_p = confi_int_p.iloc[:, 1]

In [ ]:
plt.figure(figsize = (14, 5))
plt.title("Purchase prediction by SARIMA", fontsize = 25)

plt.plot(sample_df[-24:].index, sample_df[-24:].values, label = 'Actual values', color = "blue", marker = "o")

plt.plot(mean_prediction[-24:].index, mean_prediction[-24:].values, label = 'Prediction', color = "green", marker = "o")
plt.fill_between(mean_prediction[-24:].index, lower_limits_p, upper_limits_p, alpha = 0.1, color = "green")

plt.legend(fontsize = 12, fancybox = True, shadow = True, frameon = True)
plt.ylabel('QTY', fontsize = 15)
plt.show()

In [ ]:
mape_sarima = mape(sample_df.iloc[-12:, 0], mean_prediction)
print(f"MAPE OF LSTM MODEL : {mape_sarima:.2f} %")

In [ ]:
rmse_sarima = sqrt(mean_squared_error(sample_df[-12:].values, mean_prediction.values))
print(f"RMSE OF LSTM MODEL : {rmse_sarima:.2f}")

# Results Sumamry

สรุปได้ว่าควรใช้
Best model:  SARIMA(0,1,1)(0,1,0) [6] / SARIMA(0,1,2)(0,1,0) [6]
สามารถทำ Seasonal ARIMA ได้
แต่ข้อมูลมีน้อยกว่า 24 เดือน
จึงควรปรับ Dataset ใหม่